In [3]:
!git clone https://github.com/wujx2001/QwT.git

Cloning into 'QwT'...
remote: Enumerating objects: 1263, done.
remote: Counting objects: 100% (1263/1263), done.
remote: Compressing objects: 100% (842/842), done.
remote: Total 1263 (delta 427), reused 1238 (delta 402), pack-reused 0 (from 0)
Receiving objects: 100% (1263/1263), 6.29 MiB | 28.89 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [4]:
%cd /content/QwT

/content/QwT


In [5]:
!curl -L -o ~/content/QwT/sstable-imagenet1k.zip\
  https://www.kaggle.com/api/v1/datasets/download/vitaliykinakh/stable-imagenet1k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  9.7G  100  9.7G    0     0  89.2M      0  0:01:52  0:01:52 --:--:-- 59.9M
unzip:  cannot find or open /content/QwT/stable-imagenet1k.zip, /content/QwT/stable-imagenet1k.zip.zip or /content/QwT/stable-imagenet1k.zip.ZIP.


In [39]:
!unzip /content/QwT/stable-imagenet1k.zip

Streaming output truncated to the last 5000 lines.
  inflating: imagenet1k/950_orange/000_3548.jpg  
  inflating: imagenet1k/950_orange/001_9743.jpg  
  inflating: imagenet1k/950_orange/002_4403.jpg  
  inflating: imagenet1k/950_orange/003_4858.jpg  
  inflating: imagenet1k/950_orange/004_3369.jpg  
  inflating: imagenet1k/950_orange/005_850.jpg  
  inflating: imagenet1k/950_orange/006_6693.jpg  
  inflating: imagenet1k/950_orange/007_104.jpg  
  inflating: imagenet1k/950_orange/008_9159.jpg  
  inflating: imagenet1k/950_orange/009_4437.jpg  
  inflating: imagenet1k/950_orange/010_3766.jpg  
  inflating: imagenet1k/950_orange/011_1402.jpg  
  inflating: imagenet1k/950_orange/012_9968.jpg  
  inflating: imagenet1k/950_orange/013_6972.jpg  
  inflating: imagenet1k/950_orange/014_7681.jpg  
  inflating: imagenet1k/950_orange/015_1175.jpg  
  inflating: imagenet1k/950_orange/016_50.jpg  
  inflating: imagenet1k/950_orange/017_9142.jpg  
  inflating: imagenet1k/950_orange/018_8278.jpg  
  i

In [40]:
import os, shutil, random

BASE = "/content/QwT/imagenet1k"
TRAIN_DIR = os.path.join(BASE, "train")
VAL_DIR = os.path.join(BASE, "val")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

# Detect old class folders (exclude train & val if they exist)
classes = [
    d for d in os.listdir(BASE)
    if os.path.isdir(os.path.join(BASE, d))
    and d not in ("train", "val")
]

print(f"Detected {len(classes)} classes")

for cls in classes:
    src_dir = os.path.join(BASE, cls)
    images = os.listdir(src_dir)
    random.shuffle(images)

    split = int(0.8 * len(images))
    train_dest = os.path.join(TRAIN_DIR, cls)
    val_dest = os.path.join(VAL_DIR, cls)

    os.makedirs(train_dest, exist_ok=True)
    os.makedirs(val_dest, exist_ok=True)

    # Move, not copy
    for i, img in enumerate(images):
        src = os.path.join(src_dir, img)
        dst_folder = train_dest if i < split else val_dest
        shutil.move(src, os.path.join(dst_folder, img))

    # Remove the now-empty original folder
    os.rmdir(src_dir)

print("\n✔ Done!")
print("Images were MOVED (not copied).")
print("Old class folders removed.")
print("Final structure:")
print("   imagenet1k/train/<classes>")
print("   imagenet1k/val/<classes>")


Detected 1000 classes

✔ Done!
Images were MOVED (not copied).
Old class folders removed.
Final structure:
   imagenet1k/train/<classes>
   imagenet1k/val/<classes>


In [7]:
!wget -P /content/QwT/pretrained_weights https://github.com/yhhhli/BRECQ/releases/download/v1.0/resnet18_imagenet.pth.tar

--2025-11-17 17:02:38--  https://github.com/yhhhli/BRECQ/releases/download/v1.0/resnet18_imagenet.pth.tar
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/337579284/a1434300-6b8e-11eb-80ab-1b2375c34a1e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-17T17%3A36%3A11Z&rscd=attachment%3B+filename%3Dresnet18_imagenet.pth.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-17T16%3A35%3A47Z&ske=2025-11-17T17%3A36%3A11Z&sks=b&skv=2018-11-09&sig=fbMbSmJYdmioU8VXFsz9WWmTuJZVaqPY7mgxm%2F0TGqc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzQwMDc1OSwibmJmIjoxNzYzMzk4OTU5LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZ

In [23]:
!pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-j_jaej9_/clip_b04c8d72c29f4dd28d1fd39cf203add9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-j_jaej9_/clip_b04c8d72c29f4dd28d1fd39cf203add9
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached albumentations-1.4.0-py3-none-any.whl.metadata (35 kB)
  Using cached braceexpand-0.1.7-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached calflops-0.3.2-py3-none-any.whl.metadata (28 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cache

In [22]:

import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of CUDA GPUs: {num_gpus}")
else:
    print("CUDA is not available on this system.")


Number of CUDA GPUs: 1


In [30]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [31]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [34]:
!python /content/QwT/QwT-cls-RepQ-ViT/qwt_resnet_group_conv_only_percentile.py \
    --model resnet18 \
    --data_dir /content/QwT/imagenet1k \
    --w_bits 4 \
    --a_bits 4 \
    --kernel_size 1 \
    --factor 64 \
    --start_block 0 \
    --batch_size 8 \
    --num_workers 2

Training with a single process on 1 GPUs.
Namespace(model='resnet18', data_dir='/content/QwT/imagenet1k', w_bits=4, a_bits=4, start_block=0, kernel_size=1, factor=64, batch_size=8, num_workers=2, seed=0, local_rank=0, drop_path=0.0, num_classes=1000, distributed=False, device='cuda:0', world_size=1, rank=0, log_dir='checkpoint/resnet18/QwTGroupConv/bs_8_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0', log_file='checkpoint/resnet18/QwTGroupConv/bs_8_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0/log.txt')
dataset mean : (0.485, 0.456, 0.406) & std : (0.229, 0.224, 0.225)
len of train_set : 93443    train_transform : Compose(
    RandomResizedCropAndInterpolation(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BICUBIC)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
len of eval_set : 42438    eval_transform : Compose(
    Resize(size=256, interpolation=bic

In [ ]:
!python /content/QwT/QwT-cls-RepQ-ViT/qwt_vit_and_deit.py \
    --data_dir /content/QwT/imagenet1k \
    --w_bits 4 \
    --a_bits 4 \
    --start_block 0 \
    --batch_size 8

Training with a single process on 1 GPUs.
Namespace(model='deit_tiny', data_dir='/content/QwT/imagenet1k', w_bits=4, a_bits=4, start_block=0, batch_size=8, num_workers=4, seed=0, local_rank=0, drop_path=0.0, num_classes=1000, distributed=False, device='cuda:0', world_size=1, rank=0, log_dir='checkpoint/deit_tiny/QwT/bs_8_worldsize_1_w_4_a_4_startblock_0_sed_0', log_file='checkpoint/deit_tiny/QwT/bs_8_worldsize_1_w_4_a_4_startblock_0_sed_0/log.txt')
dataset mean : (0.485, 0.456, 0.406) & std : (0.229, 0.224, 0.225)
len of train_set : 80000    train_transform : Compose(
    RandomResizedCropAndInterpolation(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BICUBIC)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
len of eval_set : 20001    eval_transform : Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
   

In [ ]:
!python /content/QwT/QwT-cls-RepQ-ViT/qwt_vit_and_deit.py \
    --model vit_small \
    --data_dir /content/QwT/imagenet1k \
    --w_bits 4 \
    --a_bits 4 \
    --start_block 0 \
    --batch_size 8

Training with a single process on 1 GPUs.
Namespace(model='vit_small', data_dir='/content/QwT/imagenet1k', w_bits=4, a_bits=4, start_block=0, batch_size=8, num_workers=4, seed=0, local_rank=0, drop_path=0.0, num_classes=1000, distributed=False, device='cuda:0', world_size=1, rank=0, log_dir='checkpoint/vit_small/QwT/bs_8_worldsize_1_w_4_a_4_startblock_0_sed_0', log_file='checkpoint/vit_small/QwT/bs_8_worldsize_1_w_4_a_4_startblock_0_sed_0/log.txt')
dataset mean : (0.5, 0.5, 0.5) & std : (0.5, 0.5, 0.5)
len of train_set : 80000    train_transform : Compose(
    RandomResizedCropAndInterpolation(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BICUBIC)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)
len of eval_set : 20001    eval_transform : Compose(
    Resize(size=248, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=(0.5, 0.

In [ ]:
!zip -r /content/QwT/logs_QWT.zip /content/QwT/checkpoint

  adding: content/QwT/checkpoint/ (stored 0%)
  adding: content/QwT/checkpoint/resnet18/ (stored 0%)
  adding: content/QwT/checkpoint/resnet18/QwTGroupConv/ (stored 0%)
  adding: content/QwT/checkpoint/resnet18/QwTGroupConv/bs_32_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0/ (stored 0%)
  adding: content/QwT/checkpoint/resnet18/QwTGroupConv/bs_32_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0/log.txt (deflated 44%)
  adding: content/QwT/checkpoint/resnet18/QwTGroupConv/bs_8_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0/ (stored 0%)
  adding: content/QwT/checkpoint/resnet18/QwTGroupConv/bs_8_worldsize_1_w_4_a_4_kernelsize_1_factor_64_startblock_0_sed_0/log.txt (deflated 71%)
  adding: content/QwT/checkpoint/deit_tiny/ (stored 0%)
  adding: content/QwT/checkpoint/deit_tiny/QwT/ (stored 0%)
  adding: content/QwT/checkpoint/deit_tiny/QwT/bs_32_worldsize_1_w_4_a_4_startblock_0_sed_0/ (stored 0%)
  adding: content/QwT/checkpoint/deit_tiny/QwT/b